Assignment 16

In [266]:
import numpy as np
import cv2
import math
import sys
import time
import enum

# byte character to binary string
def ByteToBinary(byteString):
        return '{0:4b}'.format(int(byteString, base = 16))

# Get the risk area
def GetEncodedData(dataset):
    data = []
    with open(dataset, 'r') as f:
        data = f.readlines()
    # strip the new line character
    data = [x.strip() for x in data]
    return data

class PackageTypeId(enum.Enum):
    Unknown = -1
    Sum = 0
    Product = 1
    Minimum = 2
    Maximum = 3
    LiteralValue = 4
    GreaterThan = 5
    LessThan = 6
    EqualTo = 7

class DataPackage:
    def __init__(self):
        self.Id = 0
        self.version = 0
        self.typeId = PackageTypeId.Unknown
        self.decodedData = ''
        self.SubPackages = []
    
    def __str__(self):
        return  'Id: ' + str(self.Id) + ' Type: ' + str(self.typeId) + ' Version: ' + str(self.version) + ' DecodedData: ' + str(self.decodedData) + ' SubPackages: ' + str(self.SubPackages)
    
    def __repr__(self):
        return  'Id: ' + str(self.Id) + ' Type: ' + str(self.typeId) + ' Version: ' + str(self.version) + ' DecodedData: ' + str(self.decodedData) + ' SubPackages: ' + str(self.SubPackages)
    
    

def GetPackage(i, bitStream, packagecounter = 0):
    print("package")
    dataPackage = DataPackage()
    print(bitStream[i:i+3])
    print(bitStream[i+3:i+6])
    dataPackage.Id = packagecounter
    dataPackage.version = int(bitStream[i:i+3], 2)
    dataPackage.typeId = PackageTypeId(int(bitStream[i+3:i+6], 2))
    print(dataPackage)
    dataString = ''
    i+=6

    if (dataPackage.typeId  == PackageTypeId.LiteralValue):
        
        continueWhile = True
        while (continueWhile):
            if len(bitStream) >= i+5:
                dataString += bitStream[i+1:i+5]
            else:
                raise Exception('Data stream is not valid')
            if bitStream[i] == '0':
                continueWhile = False   
            i+=5
                
        dataPackage.decodedData = int(dataString, 2)
        
    else:
        dataPackage.lengthId = int(bitStream[i], 2)
        i += 1 
        if dataPackage.lengthId == 0:
            length = int(bitStream[i:i+15], 2)
            print("Length[15]: ", length)
            i+=15
            endOfPackage = i + length
            while (i < endOfPackage):
                print("Index: ",i, " End Of Package: ", endOfPackage)
                i , subPackage, packagecounter =  GetPackage(i, bitStream,packagecounter+1)
                dataPackage.SubPackages.append(subPackage)
        else:
            
            count = int(bitStream[i:i+11], 2)
            print("Package Count[11]: ", count )
            i+=11
            
            for j in range(count):
                print("Index: ",i, " Package: ",j ," of " , count)
                i , subPackage, packagecounter=  GetPackage(i, bitStream,packagecounter+1)
                dataPackage.SubPackages.append(subPackage)

    print("Index: ", i)
                
    return i, dataPackage , packagecounter+1
    


def DeCodeDataStream(dataStream):
    bitStream = ByteToBinary(dataStream)
    extend = len(bitStream) % 8
    if extend != 0:
        bitStream = '0' * (8 - extend)+ bitStream
    DataPackages = []
    print(bitStream)
    i = 0
    while i < len(dataStream):
        print("Next Package: ", i)
        i, datapackage, _ = GetPackage(i, bitStream, 0)
        DataPackages.append(datapackage)
        if i + 10 > len(bitStream):
            break
    return DataPackages

def CountVersions(dataPackages):
    count = 0
    for dataPackage in dataPackages:
        count += dataPackage.version
        if len(dataPackage.SubPackages) > 0:
            count += CountVersions(dataPackage.SubPackages)
    return count


In [267]:

dataStreams = GetEncodedData('./data/aoc16_tests.txt')

for i in range(len(dataStreams)):
    packages = DeCodeDataStream(dataStreams[i])
    count = CountVersions(packages)

    print(count)
    print("="*20)

    

110100101111111000101000
Next Package:  0
package
110
100
Id: 0 Type: PackageTypeId.LiteralValue Version: 6 DecodedData:  SubPackages: []
Index:  21
6
00111000000000000110111101000101001010010001001000000000
Next Package:  0
package
001
110
Id: 0 Type: PackageTypeId.LessThan Version: 1 DecodedData:  SubPackages: []
Length[15]:  27
Index:  22  End Of Package:  49
package
110
100
Id: 1 Type: PackageTypeId.LiteralValue Version: 6 DecodedData:  SubPackages: []
Index:  33
Index:  33  End Of Package:  49
package
010
100
Id: 3 Type: PackageTypeId.LiteralValue Version: 2 DecodedData:  SubPackages: []
Index:  49
Index:  49
9
11101110000000001101010000001100100000100011000001100000
Next Package:  0
package
111
011
Id: 0 Type: PackageTypeId.Maximum Version: 7 DecodedData:  SubPackages: []
Package Count[11]:  3
Index:  18  Package:  0  of  3
package
010
100
Id: 1 Type: PackageTypeId.LiteralValue Version: 2 DecodedData:  SubPackages: []
Index:  29
Index:  29  Package:  1  of  3
package
100
100
Id: 

In [268]:
dataStreams = GetEncodedData('./data/aoc16.txt')

for i in range(len(dataStreams)):
    packages = DeCodeDataStream(dataStreams[i])
    count = CountVersions(packages)

    print(count)
    print("="*20)

1110001000001101010000010000000010101010100111000000000110011001110010100110101000111101100111010110001101010010001010010100110101000111101100111010110001101010010000110011010111111011111000111111110111010010010100010000000000111000011100110110010101110110000000001011010001101111100011011100011000000000101011101000000000100111001111001100110100101101010100000010100010110110011000000000101011111000000000101011001010010101100101010010010010110111001001111101100010101000110011000011110011001110111011110011010010010111000110001000110000000001011110100000000001010100011001101101101010100110111111011011001110101001011011010101100101000100110000000001010011000000000001101000011001011101010110100111001001010101110101111001100100100110111101011110011010010010000000001010000101100100110000011010011001011110001001101100100001100111110111011110011111010111111001000111100101001111111001110010111100110001000000001100000000010101100110100100001010010101001010100111000000001000101001101010010111000001

In [269]:
def GetEncodedDataWithResults(dataset):
    data = []
    with open(dataset, 'r') as f:
        data = f.readlines()
    # strip the new line character
    data = [x.strip().split(",") for x in data]
    return data

def ProcessPackages(package):

    
    if package.typeId == PackageTypeId.Sum:
        print("Sum: ", package)
        value = 0
        for subPackage in package.SubPackages:
            value += ProcessPackages(subPackage)
        return value

    elif package.typeId == PackageTypeId.Product:
        print("Product: ", package)

        value = 1
        for subPackage in package.SubPackages:
            value *= ProcessPackages(subPackage)
        return value

    elif package.typeId == PackageTypeId.Minimum:
        print("Minimum: ", package)
        value = 90000000000000
        for subPackage in package.SubPackages:
            value = min(value, ProcessPackages(subPackage))
        return value

    elif package.typeId == PackageTypeId.Maximum:
        print("Maximum: ", package)
        value = -90000000000000
        for subPackage in package.SubPackages:
            value = max(value, ProcessPackages(subPackage))
        return value

    elif package.typeId == PackageTypeId.LiteralValue:
        print("LiteralValue: ", package)
        return package.decodedData

    elif package.typeId == PackageTypeId.GreaterThan:
        print("GreaterThan: ", package)
        return int(ProcessPackages(package.SubPackages[0]) > ProcessPackages(package.SubPackages[1]))

    elif package.typeId == PackageTypeId.LessThan:
        print("LessThan: ", package)
        return int(ProcessPackages(package.SubPackages[0]) < ProcessPackages(package.SubPackages[1]))
        
    elif package.typeId == PackageTypeId.EqualTo:
        print("EqualTo: ", package)
        return int(ProcessPackages(package.SubPackages[0]) == ProcessPackages(package.SubPackages[1]))

In [270]:
dataStreams = GetEncodedDataWithResults('./data/aoc16_tests1.txt')

for i in range(len(dataStreams)):
    packages = DeCodeDataStream(dataStreams[i][0])
    print(packages)
    value = ProcessPackages(packages[0])
    print(value ,  dataStreams[i][1] , int(value)==int(dataStreams[i][1]) )
    print("="*20)
    

1100001000000000101101000000101010000010
Next Package:  0
package
110
000
Id: 0 Type: PackageTypeId.Sum Version: 6 DecodedData:  SubPackages: []
Package Count[11]:  2
Index:  18  Package:  0  of  2
package
110
100
Id: 1 Type: PackageTypeId.LiteralValue Version: 6 DecodedData:  SubPackages: []
Index:  29
Index:  29  Package:  1  of  2
package
010
100
Id: 3 Type: PackageTypeId.LiteralValue Version: 2 DecodedData:  SubPackages: []
Index:  40
Index:  40
[Id: 0 Type: PackageTypeId.Sum Version: 6 DecodedData:  SubPackages: [Id: 1 Type: PackageTypeId.LiteralValue Version: 6 DecodedData: 1 SubPackages: [], Id: 3 Type: PackageTypeId.LiteralValue Version: 2 DecodedData: 2 SubPackages: []]]
Sum:  Id: 0 Type: PackageTypeId.Sum Version: 6 DecodedData:  SubPackages: [Id: 1 Type: PackageTypeId.LiteralValue Version: 6 DecodedData: 1 SubPackages: [], Id: 3 Type: PackageTypeId.LiteralValue Version: 2 DecodedData: 2 SubPackages: []]
LiteralValue:  Id: 1 Type: PackageTypeId.LiteralValue Version: 6 Decoded

In [271]:
dataStreams = GetEncodedData('./data/aoc16.txt')

for i in range(len(dataStreams)):
    packages = DeCodeDataStream(dataStreams[i])
    print(packages)
    value = ProcessPackages(packages[0])
    print(value)
    print("="*20)

1110001000001101010000010000000010101010100111000000000110011001110010100110101000111101100111010110001101010010001010010100110101000111101100111010110001101010010000110011010111111011111000111111110111010010010100010000000000111000011100110110010101110110000000001011010001101111100011011100011000000000101011101000000000100111001111001100110100101101010100000010100010110110011000000000101011111000000000101011001010010101100101010010010010110111001001111101100010101000110011000011110011001110111011110011010010010111000110001000110000000001011110100000000001010100011001101101101010100110111111011011001110101001011011010101100101000100110000000001010011000000000001101000011001011101010110100111001001010101110101111001100100100110111101011110011010010010000000001010000101100100110000011010011001011110001001101100100001100111110111011110011111010111111001000111100101001111111001110010111100110001000000001100000000010101100110100100001010010101001010100111000000001000101001101010010111000001